### load libraries

In [1]:
library(arules)
library(arulesViz)

Loading required package: Matrix


Attaching package: 'arules'


The following objects are masked from 'package:base':

    abbreviate, write




### load data

In [2]:
df_tran <- read.transactions('arm_data.txt', format="basket", skip=1, sep=',')

In [3]:
inspect(head(df_tran))

    items                    
[1] {indie, pop}             
[2] {indie, rock, trance}    
[3] {metal, rock, trance}    
[4] {rock}                   
[5] {blues, metal, pop, rock}
[6] {pop, rock}              


In [4]:
df_tran

transactions in sparse format with
 10000 transactions (rows) and
 86 items (columns)